# Getting Started with DaskSQL
*By Winston Robson*

In this notebook, we will cover how to read and query CSV files with cuDF and DaskSQL.

# Create CUDA cluster

In [1]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

In [3]:
# create a local CUDA cluster
cluster = LocalCUDACluster()
client = Client(cluster)
client

/home/u00ubc1kg5n2YAppSj357/miniconda3/envs/rapids-dask-sql/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34581 instead
  warnings.warn(
distributed.diskutils - INFO - Found stale lock file and directory '/home/u00ubc1kg5n2YAppSj357/notebooks-contrib/getting_started_materials/hello_worlds/blazingsql/dask-worker-space/worker-s_pd4m6x', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/u00ubc1kg5n2YAppSj357/notebooks-contrib/getting_started_materials/hello_worlds/blazingsql/dask-worker-space/worker-q7vp4eo5', purging
distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.diskutils - INFO - Found stale lock file and directory '/home/u00ubc1kg5n2YAppSj357/notebooks-contrib/getting_started_materials/hello_worlds/blazingsql/dask-worker-space/worker-2hut3hc2', purging
distributed.diskutils - INFO - Found s

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:34581/status,
Dashboard: http://127.0.0.1:34581/status,Workers: 8
Total threads: 8,Total memory: 503.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43133,Workers: 8
Dashboard: http://127.0.0.1:34581/status,Total threads: 8
Started: Just now,Total memory: 503.79 GiB
Comm: tcp://127.0.0.1:44539,Total threads: 1
Dashboard: http://127.0.0.1:34065/status,Memory: 62.97 GiB
Nanny: tcp://127.0.0.1:44341,


#### Download Data
This cell will check if you have the data for this demo, and, if you don't, will download it for you.

In [2]:
import os

# relative path to data 
data_dir = '../../../data/blazingsql/'
# file name
fn = 'Music.csv'

# does folder exist?
if not os.path.exists(data_dir):
    print('creating blazingsql directory\n')
    # create folder
    os.system('mkdir ../../data/blazingsql')

# do we have music file?
if not os.path.isfile(data_dir + fn):
    !wget -P ../../../data/blazingsql 'https://s3.amazonaws.com/blazingsql-colab/Music.csv'
else:
    print("You've got the data!")

You've got the data!


## Create DaskContext
You can think of DaskContext much like a Spark Context (i.e. information such as FileSystems registered & Tables created will be stored).

In [4]:
from dask_sql import Context
# start up DaskSQL
dc = Context()

## Read CSV
First we need to download a CSV file. Then we use cuDF to read the CSV file, which gives us a GPU DataFrame (GDF). 

To learn more about the GDF and how it enables end to end workloads on rapids, check out *[The GPU DataFrame (GDF) and cuDF in RAPIDS AI](https://blog.blazingdb.com/blazingsql-part-1-the-gpu-dataframe-gdf-and-cudf-in-rapids-ai-96ec15102240)*.

In [5]:
import cudf

# cudf (gpu) dataframe from csv 
gdf = cudf.read_csv(data_dir + fn)

# let's see how it looks
gdf.head()

,ARTIST,RATING,YEAR,LOCATION,FESTIVAL_SET
0,Arcade Fire,10.0,2018.0,Las Vegas,1.0
1,Justice,10.0,2018.0,Las Vegas,1.0
2,Florence and The Machine,10.0,2018.0,Las Vegas,1.0
3,Odesza,10.0,2018.0,Indio,1.0
4,Bon Iver,10.0,2017.0,Indio,1.0


## Create a Table
Now we just need to create a table. 

In [6]:
# DaskSQL table from cuDF DataFrame 
dc.create_table('music', gdf)

## Query a Table
That's it! Now when you can write a SQL query the data will get processed on the GPU with DaskSQL, and the output will be a GPU DataFrame (GDF) inside RAPIDS!

In [7]:
# query artist, rating & location for 10 events with a rating of at least 7
result_gdf = dc.sql('SELECT ARTIST, RATING, LOCATION FROM music where RATING >= 7 LIMIT 10')

# display dataframe (type(result_gdf)==cudf.core.dataframe.DataFrame)
result_gdf

,ARTIST,RATING,LOCATION
npartitions=1,,,
0,object,float64,object
345,...,...,...


In [8]:
import time

t0 = time.time()

In [9]:
# define query 
query = '''
        select 
            ARTIST, RATING, 
            LOCATION, FESTIVAL_SET 
        from 
            music 
        where
            LOCATION = 'San Francisco'
            '''

# pull events in San Francisco, CA
gdf = dc.sql(query)


# sample 7 rows by converting to pandas 
gdf.compute().sample(7)

,ARTIST,RATING,LOCATION,FESTIVAL_SET
268,The Japanese House,4.0,San Francisco,1.0
113,Big Grams,7.0,San Francisco,1.0
51,Griz,8.0,San Francisco,1.0
235,Halsey,5.0,San Francisco,1.0
323,Two Friends,3.0,San Francisco,0.0
313,Zedd,3.0,San Francisco,1.0
140,The Kooks,7.0,San Francisco,1.0


In [10]:
t1 = time.time()
print(f"run_stuff took {t1-t0}s")

run_stuff took 0.26733851432800293s


In [12]:
len(gdf.index)

53

# You're Ready to Rock
And... thats it! You are now live with DaskSQL.


Check out our [docs](https://dask-sql.readthedocs.io/) to get fancy or to learn more about how DaskSQL works with the rest of [RAPIDS AI](https://rapids.ai/).